In [2]:
import os
import pandas as pd

In [3]:
BASE_DIR = '/mnt/f/OneDrive/文档（共享）/4_古代DNA/'

In [4]:
# 获取`/mnt/f/OneDrive/文档（共享）/4_古代DNA`文件夹下的所有的文件夹内的所有的命名为`aDNA.txt`文件，将其合并为一个DataFrame
def merge_adna_files(root_dir):
    adna_files = []
    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if file == 'aDNA.txt':
                adna_files.append(os.path.join(root, file))
    
    # 合并所有的文件
    df_list = []
    for file in adna_files:
        df = pd.read_csv(file, sep='\t', header=None)
        df_list.append(df)
    
    merged_df = pd.concat(df_list, ignore_index=True)
    return merged_df

In [5]:
merged_df = merge_adna_files(BASE_DIR)
# 列名命名为`ID`
merged_df.columns = ['ID']
merged_df
# 获取`ID`列的前缀(即数字之前的字母部分)
merged_df['Prefix'] = merged_df['ID'].str.extract(r'([a-zA-Z]+)')[0]
# 检查前缀有多少种独特的类型
unique_prefixes = merged_df['Prefix'].unique()
print(f"检测到ID中的前缀如下所示: {unique_prefixes}")
print(f"一般来说:`SRR`前缀→NCBI")
print(f"\t`SAMEA`前缀→ENA")
print(f"如果出现其他前缀，请检查数据源是否正确。")

检测到ID中的前缀如下所示: ['SRR' 'SAMEA']
一般来说:`SRR`前缀→NCBI
	`SAMEA`前缀→ENA
如果出现其他前缀，请检查数据源是否正确。


In [7]:
merged_df.drop_duplicates(subset='ID')

,ID,Prefix
0,SRR17043543,SRR
1,SRR17043542,SRR
2,SRR17043541,SRR
3,SRR17043540,SRR
4,SRR17043539,SRR
...,...,...
913,SAMEA7050474,SAMEA
914,SAMEA7050463,SAMEA
915,SAMEA7050467,SAMEA
916,SAMEA7050469,SAMEA


In [21]:
# 将不同的前缀的df分别保存到`/mnt/f/OneDrive/文档（共享）/4_古代DNA/`的不同的前缀的txt文件中
for prefix in unique_prefixes:
    prefix_df = merged_df[merged_df['Prefix'] == prefix]
    output_file = f'{BASE_DIR}/{prefix}_aDNA.txt'
    prefix_df[['ID']].to_csv(output_file, sep='\t', index=False, header=False)
    print(f"已将前缀为{prefix}的ID保存到{output_file}")

已将前缀为SRR的ID保存到/mnt/f/OneDrive/文档（共享）/4_古代DNA//SRR_aDNA.txt
已将前缀为SAMEA的ID保存到/mnt/f/OneDrive/文档（共享）/4_古代DNA//SAMEA_aDNA.txt
